In [2]:
import pandas as pd

file_to_load = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)

In [8]:
player_total = purchase_data["SN"].nunique() # Total unique number of players
print(player_total) # Print to show Player Count: total individual players counted

576


In [9]:
# Purchasing Analysis (Total)
unique_item = purchase_data["Item Name"].nunique() # Number of unique items
avg_purchase = purchase_data["Price"].mean() # Average Purchase Price
num_purchase = purchase_data["Purchase ID"].count() # Total Number of Purchases
total_revenue = purchase_data["Price"].sum() # Total Revnue

# Created Summary DataFrame
total_summary = pd.DataFrame([{'Number of Unique Items': unique_item,'Average Purchase Price':avg_purchase,
                               'Total Number of Purchases':num_purchase,'Total Revenue':total_revenue}],['Summary'])
formatter = {'Average Purchase Price':'${:,.2f}', 'Total Revenue':'${:,.2f}'} # Optional: Apply formatting
for key, i in formatter.items():
    total_summary[key] = total_summary[key].apply(i.format)
total_summary # Show Summary DataFrame within script

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
Summary,179,$3.05,780,"$2,379.77"


In [20]:
# Gender Demographics

unique_player = purchase_data.drop_duplicates("SN") # Drop Overlapping Identical Users ID
gen_count = unique_player["Gender"].value_counts() # Find count of each gender and create series
gen_percent = ((unique_player["Gender"].value_counts(normalize=True))*100).map("{:.0f}%".format) # Find % & do series
gender_demo_table = gen_count.to_frame(name="Individuals") # Turn Series to DataFrame and rename column
gen_demo_t = gender_demo_table.join(gen_percent).rename(columns={"Gender":"Percent"}) # Join % data to DataFrame
gen_demo_t # Show Gender Demongraphics DataFrame within script

,Individuals,Percent
Male,484,84%
Female,81,14%
Other / Non-Disclosed,11,2%


In [57]:
# Purchasing Analysis by Gender

pur_gen_count = purchase_data["Gender"].value_counts() # Purchase Count per Gender specific

# Create DataFrame according to each gender and sum of Total Purchase
pur_gen = purchase_data[["Gender","Price"]].groupby("Gender").sum().rename(columns={"Price":"Total Purchase Value"})
pur_gen["Purchase Count"] = pur_gen_count # Add on Purchase Count to existing DataFrame

# Add on Average Purchase Price calculated to exiting DataFrame
pur_gen["Average Purchase Price"] = ((pur_gen["Total Purchase Value"])/pur_gen["Purchase Count"]).map("${:.2f}".format)
# Add on Average Total Purchase per Person calculated to exiting DataFrame
pur_gen["Avg Total Purchase per Person"] = ((pur_gen["Total Purchase Value"])/gen_demo_t["Individuals"]).map("${:.2f}".format)
# Re-organize columns
pur_gen = pur_gen.reindex(columns=["Purchase Count","Average Purchase Price",
                                           "Total Purchase Value", "Avg Total Purchase per Person"])
pur_gen["Total Purchase Value"] = pur_gen["Total Purchase Value"].map("${:.2f}".format)
pur_gen # Show Purchasing Analysis by Gender DataFrame within script

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [53]:
bins_set = [0,9,14,19,24,29,34,39,100]
label_bin = ["<10","10-14",'15-19','20-24','25-29','30-34','35-39','40+']

#Age Demogaphics

age_group = pd.cut(unique_player["Age"], bins=bins_set, labels=label_bin) # Create bin structure of DataFrame

age_count = age_group.value_counts(sort=False) # Find Total Count per person in age group
age_per = (age_group.copy().value_counts(normalize=True, sort=False) * 100).map("{:.2f}%".format) # Find % per age group
summary_age = age_count.to_frame(name="Total Count") # Format DataFrame from existing Series data
summary_age["Percentage of Players"] = age_per # Add age percent data Series to DataFrame
summary_age # Show Age Demographics DataFrame within script

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [140]:
# Purchasing Analysis by Age

purchase_data["Age Ranges"] =  pd.cut(purchase_data["Age"], bins=bins_set, labels=label_bin) # Create and add bin to DataFrame

pur_age_count = purchase_data["Age Ranges"].value_counts(sort=False) # Purchase Count
# Total Purchase Value
pur_age = purchase_data[["Age Ranges","Price"]].groupby("Age Ranges").sum().rename(columns={"Price":"Total Purchase Value"})
pur_age["Purchase Count"] = pur_age_count # Add Purchase count to Total Purchase Value

pur_age_avg = pur_age["Total Purchase Value"]/pur_age_count # Average Purchase Price
pur_age_person = pur_age["Total Purchase Value"]/age_count # Avg Total Purchase per Person
pur_age2 = pd.DataFrame({"Average Purchase Price": pur_age_avg, # Create and join DataFrames
                        "Avg Total Purchase per Person": pur_age_person}).join(pur_age)
pur_age = pur_age2[["Purchase Count","Average Purchase Price", # Re-organize columns
                   "Total Purchase Value","Avg Total Purchase per Person"]]

formatter2 = {'Average Purchase Price':'${:,.2f}', 'Total Purchase Value':'${:,.2f}', # Apply formatting
             'Avg Total Purchase per Person':'${:,.2f}'} 
for key, i in formatter2.items():
    pur_age[key] = pur_age[key].apply(i.format)
pur_age # Show Purchasing Analysis by Age DataFrame within script

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [223]:
# Top Spenders

top_spend = purchase_data[["SN","Price"]]
top_spend = top_spend.groupby("SN").sum().rename(columns={"Price":"Total Purchase Value"})
top_spend["Purchase Count"] = purchase_data["SN"].value_counts()
top_spend["Average Purchase Price"] = top_spend["Total Purchase Value"]/top_spend["Purchase Count"]
top_spend = top_spend[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_five = top_spend.sort_values(by='Total Purchase Value', ascending=False).head()

formatter3 = {'Average Purchase Price':'${:,.2f}', 'Total Purchase Value':'${:,.2f}'} # Apply formatting
for key, i in formatter3.items():
    top_five[key] = top_five[key].apply(i.format)
top_five # Show Top Spender DataFrame within script

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [344]:
# Most Popular Items

item_count = purchase_data.groupby(["Item ID", "Item Name"])["Item Name"].count().to_frame(name="Purchase Count")
item_price = purchase_data.drop_duplicates("Item ID")
item_price = item_price[["Item ID", "Item Name","Price"]].set_index(["Item ID", "Item Name"]).rename(columns={"Price":"Item Price"})
item_data = purchase_data[["Price","Item ID", "Item Name"]].groupby(["Item ID", "Item Name"]).sum().rename(columns={"Price":"Total Purchase Value"})

item_data_all = item_count.join(item_price).join(item_data)

popular_five = item_data_all.sort_values(by="Purchase Count", ascending=False).head()
formatter4 = {'Item Price':'${:,.2f}', 'Total Purchase Value':'${:,.2f}'} # Apply formatting
for key, i in formatter4.items():
    popular_five[key] = popular_five[key].apply(i.format)
popular_five # Show Top Spender DataFrame within script

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [346]:
# Most Profitable Items

profit_five = item_data_all.sort_values("Total Purchase Value", ascending=False).head()
for key, i in formatter4.items():
    profit_five[key] = profit_five[key].apply(i.format)
profit_five

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


## Observations
##### Written Description

1. Purchases Analysis - Total / Most Popular Items / Most Profitable Items: 
    These three tables show that out of 780 item purchase data, there were only 179 individual items; which means that some items are being purchased frequently as well as likely to be the most grossing(profitable) items.
   
2. Gender Demographics / Age Demographics / Purchases Analysis - Gender - Age:
    84% of individual players identified themselves as male, where the most revenue was created. However average price per gender shows male as lowest.
    Most players (258 count) are between 20-24 and they are 44.79% of whole. This is also where most revenue by age happened.
    Item sale should target male players in 20-24 age.
    
3. Player Count / Top Spender / Most Popular Items: 
     Total 780 transactions: 576 were unique individuals, 204(35.42% of unique players) transactions were repeated purchase by previous client.
    Popular items sell and make most revenue and safe to conclude that about 35% players with purchasing experience buys again.